#### load libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, HuberRegressor
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict,cross_validate, GridSearchCV, train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import statistics
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None
pd.options.display.max_rows = None
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### load and inspect data

In [2]:
ames_df = pd.read_csv('./data/final_df.csv', index_col = 0)

# Checking final_df after generating geoCode, found duplicated observations
ames_df.shape

(2603, 179)

In [3]:
# drop duplicated observations
ames_df = ames_df.drop_duplicates(subset=['PID'],keep = 'first')
ames_df.shape

(2558, 179)

#### preprocessing

In [4]:
ames_df_use = ames_df.copy()

# convert MSSubClass to str
ames_df_use['MSSubClass'] = ames_df_use[['MSSubClass']].astype('str')

# characterize tax distr and sch distr
ames_df_use['TxD_S'] = ames_df_use[['TxD_S']].astype('str')
ames_df_use['SchD_S'] = ames_df_use[['SchD_S']].astype('str')

# convert NA's to 0s in LotFrontage and Alley
ames_df_use['LotFrontage'] = np.where(pd.isnull(ames_df_use['LotFrontage']), 0, ames_df_use['LotFrontage'])
ames_df_use['Alley'] = np.where(pd.isnull(ames_df_use['Alley']), 0, 1)

# Impute None to Nan value in columns
none_features = ['FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','Fence','BsmtQual','BsmtCond']
for feature in none_features:
    ames_df_use[feature] = ames_df_use[feature].fillna('None')

# drop rows with NA garage car / garage area
garage_NA_index = ames_df_use.loc[pd.isnull(ames_df_use['GarageArea'])].index
ames_df_use = ames_df_use.drop(garage_NA_index,axis=0)

# add in proper bathroom numbers for 918 and 2328
ames_df_use.at[918, 'num_bathroom'] = 1
ames_df_use.at[2328, 'num_bathroom'] = 3.5

# remove unnecessary columns
ames_df_use = ames_df_use.drop(
    ['PA-PreD','PA-PostD','PA-UnTyp','PA-UntNo','X1TPr_D','X1TSc_D','Rcrd_Mo','Legal_Pr','TxD_S',\
    'X2TPr_D','X2TSc_D','X1TPr_S','X1TSc_S','X2TPr_S','X2TSc_S','ISU_lat_long','address','MA_Ownr1','MA_Ownr2',\
    'MA_Line1','MA_Line2','MA_City','MA_State','address3','location2','point2','SaleCond','Source','Date',\
    'ParType','BldgNo_S','DwlgNo_S','YrBuilt','Ext1','Ext2','GLA','GarYrBlt','Cars','MA_Zip1','MA_Zip2',\
    'ZngCdPr','ZngCdSc','ZngOLPr','ZngOLSc','PA-Nmbr','PA-Strt','PA-StSfx','Inst1_No','Inst1_Yr','Inst1_Mo',\
    'Inst1TPr','TtlVal_AsrYr','ValType','OthAc_S','ImpAc_S','LndAc_S','Prop_Addr','HSTtl_D','MilVal_D',\
    'HSTtl_S','MilVal_S','GeoRefNo','Tier','Range','AcreX_S1','ClassPr_S','ClassSc_S','LndAcX1S','ImpAcX1S',\
    'Central Air','ImpAcX2S','AcreGr','AcreNt_S','ParclRel','Rcrd_Yr','address2','SaleType',\
    'latitude2','longitude2','ISU_lat','ISU_long','altitude2','Central Air',\

    'index','MiscVal','YrSold_YYYY','MoSold_MM','PoolArea','PoolQC','MiscFeature','Street','PID','Utilities',\
    'BsmtHalfBath2','FullBath','HalfBath','TtlBsmtSF','HalfBath2','BsmtFullBath','BsmtHalfBath',\
    'BsmtFinSF1','BsmtFinSF2','BsmtFinType1','BsmtFinType2','BsmtUnfSF','LowQualFinSF','BasmtFinSF2','BasmtFinSF1',\
    'GrLivArea','1stFlrSF','2ndFlrSF','BedroomAbvGr','MasVnrType','MasVnrArea','GarageArea2','PoolArea2','NmbrBRs',\
    'Neighborhood','TotalBsmtSF','GarageYrBlt','BasmtFinSF','Heating'],\
    axis=1)

print(ames_df_use.columns)
print(f'has null: {ames_df_use.columns[ames_df_use.isnull().sum() > 0]}')

Index(['SalePrice', 'Distance', 'MSSubClass', 'MSZoning', 'LotFrontage',
       'LotArea', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'CentralAir',
       'Electrical', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'Fence', 'MoSold', 'YrSold', 'SaleCondition',
       'MasVnrArea2', 'total_LivArea', 'num_bathroom', 'SchD_S'],
      dtype='object')
has null: Index(['BsmtExposure', 'Electrical'], dtype='object')


#### test-train split

In [5]:
ames_df_use.YrSold.value_counts()

2007    595
2009    576
2008    561
2006    511
2010    314
Name: YrSold, dtype: int64

In [6]:
# one way ANOVA test
SalePr_YrSold = ames_df_use[['YrSold','SalePrice']]
from scipy.stats import f_oneway
f_oneway(SalePr_YrSold.loc[SalePr_YrSold['YrSold']==2006]['SalePrice'],
         SalePr_YrSold.loc[SalePr_YrSold['YrSold']==2007]['SalePrice'],
         SalePr_YrSold.loc[SalePr_YrSold['YrSold']==2008]['SalePrice'],
         SalePr_YrSold.loc[SalePr_YrSold['YrSold']==2009]['SalePrice'],
         SalePr_YrSold.loc[SalePr_YrSold['YrSold']==2010]['SalePrice'])

F_onewayResult(statistic=0.4853231382751426, pvalue=0.7465518551782976)

#### initial testing

In [7]:
# dummify the dataset
orig_dum = pd.get_dummies(ames_df_use, drop_first=True)
print(orig_dum.shape)

# apply log()
orig_dum['SalePrice'] = np.log(orig_dum['SalePrice'])
orig_dum['total_LivArea'] = np.log(orig_dum['total_LivArea'])
orig_dum['LotFrontage'] = np.log(orig_dum['LotFrontage']+1)
orig_dum['WoodDeckSF'] = np.log(orig_dum['WoodDeckSF']+1)
orig_dum['OpenPorchSF'] = np.log(orig_dum['OpenPorchSF']+1)
orig_dum['EnclosedPorch'] = np.log(orig_dum['EnclosedPorch']+1)
orig_dum['3SsnPorch'] = np.log(orig_dum['3SsnPorch']+1)
orig_dum['ScreenPorch'] = np.log(orig_dum['ScreenPorch']+1)

(2557, 201)


In [8]:
train_orig = orig_dum[orig_dum['YrSold']!=2010]
test_orig = orig_dum[orig_dum['YrSold']==2010]
print(train_orig.shape)
print(test_orig.shape)

X_train_orig = train_orig.drop(['SalePrice','YrSold'],axis=1)
y_train_orig = train_orig['SalePrice']
X_test_orig = test_orig.drop(['SalePrice','YrSold'],axis=1)
y_test_orig = test_orig['SalePrice']

(2243, 201)
(314, 201)


In [9]:
lin_reg = LinearRegression().fit(X_train_orig, y_train_orig)
lin_reg.score(X_train_orig, y_train_orig)

x_feature = sm.add_constant(X_train_orig)

model = sm.OLS(y_train_orig, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

#X_vif = X_train_dum[pValue[pValue<0.05].index]
X_vif = X_train_orig[pValue[pValue<0.05].drop('const').index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)

0.9321469682347108

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     146.7
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:57:54   Log-Likelihood:                 1991.9
No. Observations:                2243   AIC:                            -3598.
Df Residuals:                    2050   BIC:                            -2495.
Df Model:                         192                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.71

const                    1.217777e-13
Distance                 2.247028e-08
LotArea                  3.006918e-15
OverallQual              6.107765e-66
OverallCond              7.502397e-34
YearBuilt                6.279212e-08
YearRemodAdd             3.109154e-03
TotRmsAbvGrd             1.751382e-27
Fireplaces               3.976665e-07
GarageCars               6.319923e-03
GarageArea               1.237243e-09
OpenPorchSF              1.455422e-04
ScreenPorch              2.492661e-07
MasVnrArea2              3.749355e-02
total_LivArea            3.052351e-93
num_bathroom             8.908032e-05
MSSubClass_30            2.056005e-02
MSSubClass_90            1.114662e-02
MSZoning_FV              6.386606e-13
MSZoning_I (all)         4.887759e-02
MSZoning_RH              1.870698e-05
MSZoning_RL              4.952866e-11
MSZoning_RM              5.696572e-07
LandContour_HLS          1.328972e-03
LandContour_Lvl          3.450664e-02
LandSlope_Mod            3.996790e-02
LandSlope_Se

C:\Users\aznan\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


                  feature           VIF
0                Distance  1.733094e+01
1                 LotArea  4.535944e+00
2             OverallQual  6.944906e+01
3             OverallCond  4.826939e+01
4               YearBuilt  1.614602e+04
5            YearRemodAdd  1.662665e+04
6            TotRmsAbvGrd  4.147561e+01
7              Fireplaces  3.056007e+00
8              GarageCars  4.480739e+01
9              GarageArea  3.430907e+01
10            OpenPorchSF  3.176855e+00
11            ScreenPorch  1.203835e+00
12            MasVnrArea2  2.447298e+00
13          total_LivArea  1.469745e+03
14           num_bathroom  2.381829e+01
15          MSSubClass_30  1.440441e+00
16          MSSubClass_90           inf
17            MSZoning_FV  9.381444e+00
18       MSZoning_I (all)  1.191444e+00
19            MSZoning_RH  2.137131e+00
20            MSZoning_RL  1.434239e+02
21            MSZoning_RM  2.852436e+01
22        LandContour_HLS  1.837600e+00
23        LandContour_Lvl  2.341635e+01


In [10]:
lin_reg.score(X_train_orig, y_train_orig)
lin_reg.score(X_test_orig, y_test_orig)

0.9321469682347108

0.8670913091600575

#### minimum sample/class size

In [11]:
(1.96 * statistics.stdev(ames_df_use['SalePrice']) / 20000)**2

53.87076177104101

#### feature engineering

In [12]:
def feature_eng(df):
    # calculate age of building
    df['BldgAge'] = df['YrSold'] - df['YearBuilt']
    #effectiveage: df['BldgAge'] = df['YearBuilt'] - df['YearRemodAdd']

    # binarize YearRemodAdd
    df['Remodeled'] = np.where(df['YearRemodAdd'] == df['YearBuilt'], 0, 1)

    # binarize MSSubClass to PUD or not PUD
    df['IsPUD'] = np.where(df['MSSubClass'].isin(['120','150','160','180']), 1, 0)
    
    # binarize LotShape to Reg/not Reg
    df['LotIsReg'] = np.where(df['LotShape']=='Reg', 1, 0)
    
    # binarize LandContour to HLS and Low vs not HLS or Low
    df['HillORDepr'] = np.where(df['LandContour'].isin(['HLS','Low']), 1, 0)
    
    # binarize Condition1/2 to positive feature or no positive feature
    df['PosFeat'] = np.where(df['Condition1'].isin(['PosN','PosA'])|df['Condition2'].isin(['PosN','PosA']), 1, 0)
    
    # combine exterior material 1/2 to one column
    df['ExtMatl'] = np.where((df['Exterior1st']==df['Exterior2nd']),df['Exterior1st'], 'Mixed')
    
    # simply qual/cond features
    for col in ['ExterQual','BsmtQual','KitchenQual','FireplaceQu','GarageQual','ExterCond',\
                'BsmtCond','GarageCond','HeatingQC']:
        df[col+'_num'] = df[col].replace(['Ex','Gd','TA','Fa','Po','None'],[10,8,6,4,2,0])

    # sum up porch/housefront area
    df['TotalPorchSF'] = df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] +\
                                df['ScreenPorch'] + df['WoodDeckSF']
    
    # binarize fences to having a fence or not having a fence
    df['HasFence'] = np.where(df['Fence']=='None', 0, 1)

    # simplify Functional to 3 classes
    df['Funct_3'] = df['Functional'].replace(['Maj1', 'Maj2', 'Min1', 'Min2', 'Mod', 'Sal', 'Typ'],\
                                             ['ModToSev','ModToSev','Minor','Minor','ModToSev','ModToSev','Normal'])
    
    # drop the original columns or unused columns
    df = df.drop(['MSSubClass','YearBuilt','YearRemodAdd','LotFrontage','LotArea','LotConfig','LandSlope',\
                  'Condition1','Condition2','Exterior1st','Exterior2nd','LotShape','LandContour',\
                  'OpenPorchSF','3SsnPorch','EnclosedPorch','ScreenPorch','WoodDeckSF',\
                  'Fence','Functional','ExterQual','BsmtQual','KitchenQual','FireplaceQu',\
                  'GarageQual','ExterCond','BsmtCond','GarageCond','HeatingQC',\
                  'GarageCars','RoofStyle','RoofMatl','KitchenAbvGr'],\
                 axis = 1)
    return df

In [13]:
ames_df_eng = feature_eng(ames_df_use)

In [14]:
ames_df_eng.columns

Index(['SalePrice', 'Distance', 'MSZoning', 'Alley', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'Foundation', 'BsmtExposure',
       'CentralAir', 'Electrical', 'TotRmsAbvGrd', 'Fireplaces', 'GarageType',
       'GarageFinish', 'GarageArea', 'PavedDrive', 'MoSold', 'YrSold',
       'SaleCondition', 'MasVnrArea2', 'total_LivArea', 'num_bathroom',
       'SchD_S', 'BldgAge', 'Remodeled', 'IsPUD', 'LotIsReg', 'HillORDepr',
       'PosFeat', 'ExtMatl', 'ExterQual_num', 'BsmtQual_num',
       'KitchenQual_num', 'FireplaceQu_num', 'GarageQual_num', 'ExterCond_num',
       'BsmtCond_num', 'GarageCond_num', 'HeatingQC_num', 'TotalPorchSF',
       'HasFence', 'Funct_3'],
      dtype='object')

#### dummification and log transformation

In [15]:
# dummify the dataset
ames_df_dum = pd.get_dummies(ames_df_eng, drop_first=True)
print(ames_df_dum.shape)
print(ames_df_dum.shape)

# apply log()
ames_df_dum['SalePrice'] = np.log(ames_df_dum['SalePrice'])
ames_df_dum['total_LivArea'] = np.log(ames_df_dum['total_LivArea'])
#ames_df_dum['Distance'] = np.log(X_train_dum['Distance'])
ames_df_dum['TotalPorchSF'] = np.log(ames_df_dum['TotalPorchSF']+1)

(2557, 89)
(2557, 89)


In [16]:
train = ames_df_dum[ames_df_dum['YrSold']!=2010]
test = ames_df_dum[ames_df_dum['YrSold']==2010]
print(train.shape)
print(test.shape)

X_train = train.drop(['SalePrice','YrSold'],axis=1)
y_train = train['SalePrice']
X_test = test.drop(['SalePrice','YrSold'],axis=1)
y_test = test['SalePrice']

(2243, 89)
(314, 89)


#### linear regression, p-values, and VIFs

In [17]:
lin_reg = LinearRegression().fit(X_train, y_train)
lin_reg.score(X_train, y_train)

x_feature = sm.add_constant(X_train)

model = sm.OLS(y_train, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

#X_vif = X_train_dum[pValue[pValue<0.05].index]
X_vif = X_train[pValue[pValue<0.05].drop('const').index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)

0.9170440020274087

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     273.8
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:57:55   Log-Likelihood:                 1766.5
No. Observations:                2243   AIC:                            -3357.
Df Residuals:                    2155   BIC:                            -2854.
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     8.27

const                     0.000000e+00
Distance                  2.039749e-05
OverallQual               9.004325e-74
OverallCond               3.116463e-41
TotRmsAbvGrd              5.627140e-33
Fireplaces                3.621850e-09
GarageArea                5.232993e-32
total_LivArea            2.016258e-101
num_bathroom              8.279832e-05
BldgAge                   1.790871e-06
LotIsReg                  2.159964e-02
HillORDepr                5.137700e-06
ExterQual_num             1.384016e-02
BsmtQual_num              7.967216e-07
KitchenQual_num           6.237743e-06
HeatingQC_num             2.090769e-04
TotalPorchSF              9.436876e-03
MSZoning_FV               7.411762e-12
MSZoning_I (all)          4.612259e-02
MSZoning_RH               6.659612e-04
MSZoning_RL               2.871565e-09
MSZoning_RM               2.604717e-05
BldgType_2fmCon           4.027566e-03
BldgType_Duplex           9.304687e-10
BldgType_Twnhs            1.271284e-02
HouseStyle_2Story        

                  feature         VIF
0                Distance   16.625646
1             OverallQual   71.878423
2             OverallCond   40.513198
3            TotRmsAbvGrd   39.758986
4              Fireplaces    2.791724
5              GarageArea   10.492983
6           total_LivArea  450.866869
7            num_bathroom   21.242537
8                 BldgAge   12.714066
9                LotIsReg    3.305161
10             HillORDepr    1.205353
11          ExterQual_num  106.523024
12           BsmtQual_num   39.163275
13        KitchenQual_num   72.625019
14          HeatingQC_num   33.377068
15           TotalPorchSF    7.133007
16            MSZoning_FV    8.012833
17       MSZoning_I (all)    1.138967
18            MSZoning_RH    1.933441
19            MSZoning_RL  122.937397
20            MSZoning_RM   24.989622
21        BldgType_2fmCon    1.180660
22        BldgType_Duplex    1.371554
23         BldgType_Twnhs    1.290013
24      HouseStyle_2Story    2.243255
25        Ho

#### train and test error

In [18]:
lin_reg.score(X_train, y_train)
lin_reg.score(X_test, y_test)

0.9170440020274087

0.8548036170140827

#### lasso (use the original features)

In [19]:
ames_df_use.columns

Index(['SalePrice', 'Distance', 'MSSubClass', 'MSZoning', 'LotFrontage',
       'LotArea', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'CentralAir',
       'Electrical', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'Fence', 'MoSold', 'YrSold', 'SaleCondition',
       'MasVnrArea2', 'total_LivArea', 'num_bathroom', 'SchD_S', 'BldgAge',
       'Remodeled', 'IsPUD', 'LotIsReg', 'HillORDepr', 'PosFeat', 'ExtMatl',
       'ExterQual_num', 'BsmtQua

In [20]:
# dummify the dataset
lasso_dum = pd.get_dummies(ames_df_use, drop_first=True)
print(lasso_dum.shape)

# apply log()
lasso_dum['SalePrice'] = np.log(lasso_dum['SalePrice'])
lasso_dum['total_LivArea'] = np.log(lasso_dum['total_LivArea'])
lasso_dum['LotFrontage'] = np.log(lasso_dum['LotFrontage']+1)
lasso_dum['WoodDeckSF'] = np.log(lasso_dum['WoodDeckSF']+1)
#lasso_dum['TotalPorchSF'] = np.log(lasso_dum['TotalPorchSF']+1)
lasso_dum['OpenPorchSF'] = np.log(lasso_dum['OpenPorchSF']+1)
lasso_dum['EnclosedPorch'] = np.log(lasso_dum['EnclosedPorch']+1)
lasso_dum['3SsnPorch'] = np.log(lasso_dum['3SsnPorch']+1)
lasso_dum['ScreenPorch'] = np.log(lasso_dum['ScreenPorch']+1)

(2557, 232)


In [21]:
train_lasso = lasso_dum[lasso_dum['YrSold']!=2010]
test_lasso = lasso_dum[lasso_dum['YrSold']==2010]
print(train_lasso.shape)
print(test_lasso.shape)

X_train_lasso = train_lasso.drop(['SalePrice','YrSold'],axis=1)
y_train_lasso = train_lasso['SalePrice']
X_test_lasso = test_lasso.drop(['SalePrice','YrSold'],axis=1)
y_test_lasso = test_lasso['SalePrice']

(2243, 232)
(314, 232)


In [22]:
# best best fit alpha
reg_lasso_cv = LassoCV(normalize = True, max_iter = 10000)
reg_lasso_cv.fit(X_train_lasso, y_train_lasso)
reg_lasso_cv.alpha_

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=10000, n_alphas=100, n_jobs=None, normalize=True,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

3.790330532229508e-05

In [23]:
# put the alpha value back in the original lasso
reg_lasso = Lasso(normalize=True)
reg_lasso.set_params(alpha = reg_lasso_cv.alpha_)
reg_lasso.fit(X_train_lasso, y_train_lasso)
lasso_coef_df = pd.DataFrame({'feature':X_train_lasso.columns,
                              'coef':reg_lasso.coef_})

num_feat = len(lasso_coef_df[lasso_coef_df['coef']!=0])
print(f'{num_feat} non-zero features')

Lasso(alpha=3.790330532229508e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=3.790330532229508e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

107 non-zero features


#### test and train errors

In [24]:
print(reg_lasso.score(X_train_lasso, y_train_lasso))
reg_lasso.score(X_test_lasso, y_test_lasso)

0.9276405056078176


0.861987763846034

#### lasso (using engineered features)

In [25]:
X_train_lasso2 = X_train.copy()
y_train_lasso2 = y_train.copy()
X_test_lasso2 = X_test.copy()
y_test_lasso2 = y_test.copy()

In [26]:
reg_lasso_cv = LassoCV(normalize = True, max_iter = 10000)
reg_lasso_cv.fit(X_train_lasso2, y_train_lasso2)
reg_lasso_cv.alpha_

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=10000, n_alphas=100, n_jobs=None, normalize=True,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

3.296636199998347e-05

In [27]:
# put the alpha value back in the original lasso
reg_lasso = Lasso(normalize=True)
reg_lasso.set_params(alpha = reg_lasso_cv.alpha_)
reg_lasso.fit(X_train_lasso2, y_train_lasso2)
lasso_coef_df = pd.DataFrame({'feature':X_train_lasso2.columns,
                              'coef':reg_lasso.coef_})

num_feat = len(lasso_coef_df[lasso_coef_df['coef']!=0])
print(f'{num_feat} non-zero features')

Lasso(alpha=3.296636199998347e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=3.296636199998347e-05, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=True, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

61 non-zero features


In [28]:
print(reg_lasso.score(X_train_lasso2, y_train_lasso2))
reg_lasso.score(X_test_lasso2, y_test_lasso2)

0.914753530927368


0.8503637291847466

#### ridge (using original features)

In [29]:
X_train_ridge = X_train_lasso.copy()
y_train_ridge = y_train_lasso.copy()
X_test_ridge = X_test_lasso.copy()
y_test_ridge = y_test_lasso.copy()

In [30]:
alphas_ridge = np.linspace(1e-5,10,200).tolist()

In [31]:
# best best fit alpha
reg_ridge_cv = RidgeCV(normalize = True, alphas = alphas_ridge)
reg_ridge_cv.fit(X_train_ridge, y_train_ridge)
reg_ridge_cv.alpha_

RidgeCV(alphas=array([1.00000000e-05, 5.02612060e-02, 1.00512412e-01, 1.50763618e-01,
       2.01014824e-01, 2.51266030e-01, 3.01517236e-01, 3.51768442e-01,
       4.02019648e-01, 4.52270854e-01, 5.02522060e-01, 5.52773266e-01,
       6.03024472e-01, 6.53275678e-01, 7.03526884e-01, 7.53778090e-01,
       8.04029296e-01, 8.54280503e-01, 9.04531709e-01, 9.54782915e-01,
       1.00503412e+00, 1.05528533e+0...
       9.04522709e+00, 9.09547829e+00, 9.14572950e+00, 9.19598070e+00,
       9.24623191e+00, 9.29648312e+00, 9.34673432e+00, 9.39698553e+00,
       9.44723673e+00, 9.49748794e+00, 9.54773915e+00, 9.59799035e+00,
       9.64824156e+00, 9.69849276e+00, 9.74874397e+00, 9.79899518e+00,
       9.84924638e+00, 9.89949759e+00, 9.94974879e+00, 1.00000000e+01]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=True,
        scoring=None, store_cv_values=False)

0.05026120603015076

In [32]:
reg_ridge = Ridge(normalize=True, alpha = reg_ridge_cv.alpha_)
reg_ridge.fit(X_train_ridge, y_train_ridge)

Ridge(alpha=0.05026120603015076, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)

In [33]:
len(X_train_ridge.columns)

230

In [34]:
print(reg_ridge.score(X_train_ridge, y_train_ridge))
reg_ridge.score(X_test_ridge, y_test_ridge)

0.9309939854061726


0.8625179354969412

#### ridge (using engineered features)

In [35]:
X_train_ridge2 = X_train.copy()
y_train_ridge2 = y_train.copy()
X_test_ridge2 = X_test.copy()
y_test_ridge2 = y_test.copy()

In [36]:
# best best fit alpha
reg_ridge_cv = RidgeCV(normalize = True, alphas = alphas_ridge)
reg_ridge_cv.fit(X_train_ridge2, y_train_ridge2)
reg_ridge_cv.alpha_

RidgeCV(alphas=array([1.00000000e-05, 5.02612060e-02, 1.00512412e-01, 1.50763618e-01,
       2.01014824e-01, 2.51266030e-01, 3.01517236e-01, 3.51768442e-01,
       4.02019648e-01, 4.52270854e-01, 5.02522060e-01, 5.52773266e-01,
       6.03024472e-01, 6.53275678e-01, 7.03526884e-01, 7.53778090e-01,
       8.04029296e-01, 8.54280503e-01, 9.04531709e-01, 9.54782915e-01,
       1.00503412e+00, 1.05528533e+0...
       9.04522709e+00, 9.09547829e+00, 9.14572950e+00, 9.19598070e+00,
       9.24623191e+00, 9.29648312e+00, 9.34673432e+00, 9.39698553e+00,
       9.44723673e+00, 9.49748794e+00, 9.54773915e+00, 9.59799035e+00,
       9.64824156e+00, 9.69849276e+00, 9.74874397e+00, 9.79899518e+00,
       9.84924638e+00, 9.89949759e+00, 9.94974879e+00, 1.00000000e+01]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=True,
        scoring=None, store_cv_values=False)

0.05026120603015076

In [37]:
# put the alpha value back in the original ridge
reg_ridge = Ridge(normalize=True, alpha = reg_ridge_cv.alpha_)
reg_ridge.fit(X_train_ridge2, y_train_ridge2)

Ridge(alpha=0.05026120603015076, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)

In [38]:
len(X_train_ridge2.columns)

87

#### test and train errors

In [39]:
print(reg_ridge.score(X_train_ridge2, y_train_ridge2))
reg_ridge.score(X_test_ridge2, y_test_ridge2)

0.9155095992791245


0.8511679340429521

#### elastic net

In [40]:
X_train_elnt = X_train_orig.copy()
y_train_elnt = y_train_orig.copy()
X_test_elnt = X_test_orig.copy()
y_test_elnt = y_test_orig.copy()

In [41]:
reg_elasticnet_cv = ElasticNetCV(l1_ratio = 0.5, cv=5, n_alphas = 200, normalize = True)
reg_elasticnet_cv.fit(X_train_elnt, y_train_elnt)

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=1000, n_alphas=200, n_jobs=None,
             normalize=True, positive=False, precompute='auto',
             random_state=None, selection='cyclic', tol=0.0001, verbose=0)

In [42]:
print(reg_elasticnet_cv.alpha_)
print(reg_elasticnet_cv.l1_ratio_)

3.8856887089111635e-05
0.5


In [43]:
reg_elasticnet = ElasticNet(alpha = reg_elasticnet_cv.alpha_, normalize = True)
reg_elasticnet.fit(X_train_elnt, y_train_elnt)

ElasticNet(alpha=3.8856887089111635e-05, copy_X=True, fit_intercept=True,
           l1_ratio=0.5, max_iter=1000, normalize=True, positive=False,
           precompute=False, random_state=None, selection='cyclic', tol=0.0001,
           warm_start=False)

In [44]:
elnt_coef_df = pd.DataFrame({'feature':X_train_elnt.columns,
                              'coef':reg_elasticnet.coef_})

num_feat = len(elnt_coef_df[elnt_coef_df['coef']!=0])
print(f'{num_feat} non-zero features')

152 non-zero features


In [45]:
print(reg_elasticnet.score(X_train_elnt, y_train_elnt))
reg_elasticnet.score(X_test_elnt, y_test_elnt)

0.9276867925302981


0.8595075258577324

In [46]:
ratio = np.linspace(0.01,1,20)
reg_elasticnet_cv = ElasticNetCV(l1_ratio = ratio, cv=5, n_alphas = 200, normalize = True)
reg_elasticnet_cv.fit(X_train_elnt, y_train_elnt)
print(reg_elasticnet_cv.alpha_)
print(reg_elasticnet_cv.l1_ratio_)

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
             l1_ratio=array([0.01      , 0.06210526, 0.11421053, 0.16631579, 0.21842105,
       0.27052632, 0.32263158, 0.37473684, 0.42684211, 0.47894737,
       0.53105263, 0.58315789, 0.63526316, 0.68736842, 0.73947368,
       0.79157895, 0.84368421, 0.89578947, 0.94789474, 1.        ]),
             max_iter=1000, n_alphas=200, n_jobs=None, normalize=True,
             positive=False, precompute='auto', random_state=None,
             selection='cyclic', tol=0.0001, verbose=0)

2.655317157294313e-05
1.0


#### huber regression

In [47]:
X_train_huber = X_train.copy()
y_train_huber = y_train.copy()
X_test_huber = X_test.copy()
y_test_huber = y_test.copy()

In [48]:
reg_huber = HuberRegressor(max_iter = 30000)
reg_huber.fit(X_train_huber, y_train_huber)

C:\Users\aznan\anaconda3\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=30000,
               tol=1e-05, warm_start=False)

In [49]:
huber_coef_df = pd.DataFrame({'feature':X_train_huber.columns,
                              'coef':reg_huber.coef_})
#huber_coef_df

In [50]:
print(reg_huber.score(X_train_huber, y_train_huber))
reg_huber.score(X_test_huber, y_test_huber)

0.9137184312531125


0.852189800177577

#### deceision tree

In [51]:
from sklearn import ensemble
from sklearn import tree

In [52]:
tree_model = tree.DecisionTreeRegressor(criterion = 'mse', max_depth = 5)
RF = ensemble.RandomForestRegressor()
BAgg = ensemble.BaggingClassifier()

In [53]:
X_train_tree = X_train.copy()
y_train_tree = y_train.copy()
X_test_tree = X_test.copy()
y_test_tree = y_test.copy()

In [54]:
tree_model.set_params()
print(tree_model.fit(X_train_tree, y_train_tree))
print(tree_model.score(X_test_tree, y_test_tree))

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')
0.75588709626496


In [55]:
# grid search

#### random forest

In [56]:
RF.set_params(oob_score=True, random_state=108)
RF.fit(X_train_tree, y_train_tree)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=108, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=108, verbose=0, warm_start=False)

In [57]:
RF.oob_score_

0.890137394521988

In [ ]:
forest_grid_params = [{
    "n_estimators": [25, 50, 100],
    "criterion": ['mse','friedman_mse','mae'],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int),
    "random_state": [108]}]
RF_grid = GridSearchCV(RF, forest_grid_params, scoring='r2', cv=5, n_jobs=-1)
%time RF_grid.fit(X_train, y_train)

In [ ]:
# Decision Tree Model
tree_reg = DecisionTreeRegressor(max_depth=10).fit(X_train1,y_train1)
print(f'R^2 of Train set: {tree_reg.score(X_train1,y_train1)}')
print(f'R^2 Test set: {tree_reg.score(X_test1,y_test1)}')



In [ ]:
# Random Forest Model
forest_reg = RandomForestRegressor(n_estimators=100,max_features=5).fit(X_train1,y_train1)
print(f'R^2 of Train set: {forest_reg.score(X_train1,y_train1)}')
print(f'R^2 Test set: {forest_reg.score(X_test1,y_test1)}')

In [ ]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# housing_prediction = lin_reg.predict(X_test)
# mean_squared_error(y_train,y_test)
# lin_reg2 = LinearRegression().fit(X_train,y_train)
# lin_reg.r2_score